In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import subprocess
import time
import psycopg2
import gcp_fx
import pandas as pd

Within console go to Cloud SQL
- create postgres instance and admin user "postgres" 
- add app account to Authenticated IAM. Note postgres considers APP_ACCOUNT to be the text left of "."

In [109]:
def exec_sql(sql):
    with gcp_fx.connect_as_postgres() as con:
        cur=con.cursor()
        cur.execute(sql)
        cur.close()

destination_table = 'pubsub_messages'
app_account = gcp_fx.cp['POSTGRES']['app_account']
exec_sql(f'grant all privileges on database postgres to "{app_account}";')
exec_sql(f'drop table if exists {destination_table};')
exec_sql(f'create table {destination_table} (eventid varchar(255), received timestamp, msg varchar(255));')
exec_sql(f'grant all privileges on all tables in schema public to "{app_account}";')

In [39]:
topic = 'cloud-run-postgres'  # to trigger cloud run fx
def run(cmd=list):
    return subprocess.run(cmd,shell=True)

In [ ]:
run(['gcloud','pubsub','topics','create',topic])  

CompletedProcess(args=['gcloud', 'pubsub', 'topics', 'create', 'cloud-run-postgres'], returncode=0)

In [80]:
# create / update cloud run fx using gen 1 functions
fx_name='cloud-run-postgres'
fx_region = 'us-east4'
source_code_path = 'cloud_run_postgres_src'

cmd = ['gcloud','functions','deploy', fx_name, f'--trigger-topic={fx_name}',
    '--runtime=python311',f'--source={source_code_path}','--entry-point=entry_point','--no-gen2',
    '--timeout=120s',f'--region={fx_region}']
# print(' '.join(cmd))
run(cmd)

#gen 2 would require event arc, setting sql cloud instance on the container, would be a good clould build project

CompletedProcess(args=['gcloud', 'functions', 'deploy', 'cloud-run-postgres', '--trigger-topic=cloud-run-postgres', '--runtime=python311', '--source=cloud_run_postgres_src', '--entry-point=entry_point', '--no-gen2', '--timeout=120s', '--region=us-east4'], returncode=0)

In [110]:
def qry_postgres():    
    import warnings
    warnings.simplefilter(action='ignore', category=UserWarning)  #sqlalchemy preferred
    with gcp_fx.connect_as_postgres() as con:
        df = pd.read_sql(f'select * from {destination_table}',con)
    print(f'{destination_table} has {df.shape[0]} records')
    return df
        
qry_postgres()

pubsub_messages has 0 records


,eventid,received,msg


In [111]:
gcp_fx.Pubs().send(topic,'initial insert')


In [112]:
qry_postgres()

pubsub_messages has 1 records


,eventid,received,msg
0,13263265980284286,2024-12-17 00:13:11.397636,initial insert


In [113]:
for i in range(3):
    gcp_fx.Pubs().send(topic,f'msg {i}')

qry_postgres()

pubsub_messages has 3 records


,eventid,received,msg
0,13263265980284286,2024-12-17 00:13:11.397636,initial insert
1,13263255917841118,2024-12-17 00:14:02.567580,msg 0
2,13263242181189612,2024-12-17 00:14:03.670921,msg 1


In [114]:
# a bit of delay, could time.sleep
qry_postgres()

pubsub_messages has 4 records


,eventid,received,msg
0,13263265980284286,2024-12-17 00:13:11.397636,initial insert
1,13263255917841118,2024-12-17 00:14:02.567580,msg 0
2,13263242181189612,2024-12-17 00:14:03.670921,msg 1
3,13263266462962500,2024-12-17 00:14:04.873005,msg 2
